 # Simplificando pandas, lectura y escritura de archivos III
 ## Lectura y manipulación de archivos excel.

 ### Importamos las librerias a utilizar

In [1]:
import os
from pathlib import Path
import glob
import numpy as np
import pandas as pd
from openpyxl import load_workbook
import xlwings as xw

 # Módulo pathlib

In [2]:
# directorio actual
dir_prin = Path.cwd()
# directorio con los datos
dir_entra = dir_prin/'entradas'
# directorio con los datos intermedios
dir_inter = dir_prin /'intermedios'
# directorio con los datos finales
dir_salid = dir_prin / 'salidas'

 ## Lectura de una sola pestaña
 ### Pestaña por defecto

In [3]:
filename = dir_entra / 'datos_clima.xlsx'
datos = pd.read_excel(filename)
datos.head()

,time,precipIntensity,temperature,apparentTemperature,dewPoint,humidity,pressure,windSpeed,cloudCover,uvIndex,visibility
0,2018-01-01 06:00:00,0.0,22.44,22.73,18.10,0.77,NaN,0.0,0.19,0.0,10.01
1,2018-01-01 07:00:00,0.0,21.88,22.15,17.75,0.77,NaN,0.0,NaN,0.0,10.01
2,2018-01-01 08:00:00,0.0,21.33,21.63,17.94,0.81,NaN,0.0,NaN,0.0,10.01
3,2018-01-01 09:00:00,0.0,20.77,21.07,17.75,0.83,NaN,0.0,NaN,0.0,10.01
4,2018-01-01 10:00:00,0.0,19.11,19.33,16.79,0.86,NaN,0.0,NaN,0.0,10.01


 ### Escogiendo la pestaña

In [4]:
datos = pd.read_excel(filename, sheet_name='febrero') 
datos.head()

,time,precipIntensity,temperature,apparentTemperature,dewPoint,humidity,pressure,windSpeed,cloudCover,uvIndex,visibility
0,2018-02-01 06:00:00,0,21.13,21.14,15.54,0.70,NaN,1.38,NaN,0.0,10.01
1,2018-02-01 07:00:00,0,21.63,21.67,15.86,0.70,NaN,3.23,NaN,0.0,10.01
2,2018-02-01 08:00:00,0,21.33,21.36,15.75,0.71,NaN,1.75,NaN,0.0,10.01
3,2018-02-01 09:00:00,0,20.58,20.58,15.42,0.72,NaN,3.12,NaN,0.0,10.01
4,2018-02-01 10:00:00,0,20.22,20.27,15.74,0.76,NaN,1.61,NaN,0.0,10.01


 ## Lectura de varias pestañas de un mismo archivo
 ### Primero conozcamos las pestañas del archivo

 Ocupamos openpyxl

 Creamos el objeto wb con la información del archivo

In [5]:
wb = load_workbook(filename)

 Extraemos el nombre de las pestañas

In [6]:
meses = wb.sheetnames
meses

['enero',
 'febrero',
 'marzo',
 'abril',
 'mayo',
 'junio',
 'julio',
 'agosto',
 'septiembre',
 'octubre',
 'noviembre',
 'diciembre']

 ### Dataframe vacío

In [7]:
datos = pd.DataFrame({'time':[],'precipIntensity':[],
                    'temperature':[],'apparentTemperature':[],
                    'dewPoint':[],'humidity':[],'pressure':[],
                    'windSpeed':[],'cloudCover':[]})
columnas = ['time','precipIntensity','temperature','apparentTemperature',
            'dewPoint','humidity','pressure','windSpeed','cloudCover']

In [8]:
for mes in meses:
    datos_temp = pd.read_excel(filename, sheet_name=mes)
    # Seleccionamos las columnas de interes
    datos_temp = datos_temp[columnas]
    # Unimos Dataframnes
    datos = pd.concat([datos, datos_temp], join='inner', axis=0)
    datos = datos[columnas]
datos = datos.set_index('time')

 ### Revisemos los datos

In [9]:
datos.head()

,precipIntensity,temperature,apparentTemperature,dewPoint,humidity,pressure,windSpeed,cloudCover
time,,,,,,,,
2018-01-01 06:00:00,0.0,22.44,22.73,18.10,0.77,NaN,0.0,0.19
2018-01-01 07:00:00,0.0,21.88,22.15,17.75,0.77,NaN,0.0,NaN
2018-01-01 08:00:00,0.0,21.33,21.63,17.94,0.81,NaN,0.0,NaN
2018-01-01 09:00:00,0.0,20.77,21.07,17.75,0.83,NaN,0.0,NaN
2018-01-01 10:00:00,0.0,19.11,19.33,16.79,0.86,NaN,0.0,NaN


In [10]:
datos.tail()

,precipIntensity,temperature,apparentTemperature,dewPoint,humidity,pressure,windSpeed,cloudCover
time,,,,,,,,
2019-01-01 01:00:00,NaN,21.59,22.18,20.00,0.91,NaN,NaN,NaN
2019-01-01 02:00:00,NaN,22.12,22.44,18.30,0.79,NaN,3.60,NaN
2019-01-01 03:00:00,NaN,21.13,21.53,18.61,0.86,NaN,1.66,NaN
2019-01-01 04:00:00,NaN,21.13,21.49,18.30,0.84,NaN,4.96,0.0
2019-01-01 05:00:00,NaN,20.13,20.57,18.61,0.91,NaN,3.60,0.0


In [11]:
datos.shape

(8642, 8)

 ## Selección de una columna
 Veremos tres formas de seleccionar una columna:
 ### Usando corchetes y nombre entre comillas

In [12]:
datos['precipIntensity'].describe()

count    3361.000000
mean        0.027658
std         0.368538
min         0.000000
25%         0.000000
50%         0.000000
75%         0.000000
max        11.805900
Name: precipIntensity, dtype: float64

 ### Usando notación de punto

In [13]:
datos.precipIntensity.describe()

count    3361.000000
mean        0.027658
std         0.368538
min         0.000000
25%         0.000000
50%         0.000000
75%         0.000000
max        11.805900
Name: precipIntensity, dtype: float64

 ### Usando "iloc" y el número de la columna

In [14]:
datos.iloc[:,0].describe()

count    3361.000000
mean        0.027658
std         0.368538
min         0.000000
25%         0.000000
50%         0.000000
75%         0.000000
max        11.805900
Name: precipIntensity, dtype: float64

 ### Otras funciones estadísticas

In [15]:
[datos['precipIntensity'].sum(), # Suma del valor de las columnas
 datos['precipIntensity'].nunique(), # Número de valores únicos
 datos['precipIntensity'].isnull().sum(), # Total de valores nulos
 datos['precipIntensity'].notnull().sum() # Total de valores no nulos
 ] 

[92.95899999999999, 57, 5281, 3361]

In [16]:
datos['precipIntensity'].value_counts()

0.0000     3286
0.1270        4
0.1930        3
0.0635        3
0.0864        3
0.3048        2
0.3175        2
1.0135        2
0.2134        2
0.3023        2
0.2032        2
0.0432        2
0.4318        2
0.2972        2
0.2565        2
3.8125        1
0.3708        1
1.6129        1
0.0838        1
8.1839        1
0.2540        1
2.3952        1
3.3045        1
2.0472        1
0.4547        1
0.2896        1
0.0610        1
0.2438        1
1.2700        1
2.6848        1
0.5715        1
1.6485        1
1.9634        1
2.2225        1
3.7592        1
0.2083        1
0.1245        1
4.0056        1
0.0406        1
1.3995        1
10.1422       1
11.8059       1
0.3835        1
0.9906        1
0.6985        1
3.6830        1
1.2040        1
1.0211        1
1.7475        1
0.8077        1
0.0279        1
5.2426        1
0.4953        1
0.1524        1
0.9093        1
0.1829        1
2.1488        1
Name: precipIntensity, dtype: int64

In [17]:
type(datos['precipIntensity'])

pandas.core.series.Series

 ## Selección de múltiples columnas
 Veremos dos formas de seleccionar una columna:
 ### Usando corchetes y nombre entre comillas

In [18]:
datos[['precipIntensity','temperature','apparentTemperature']].describe()

,precipIntensity,temperature,apparentTemperature
count,3361.000000,8640.000000,8640.000000
mean,0.027658,24.535807,25.664956
std,0.368538,3.388262,4.159469
min,0.000000,15.600000,15.640000
25%,0.000000,21.730000,22.420000
50%,0.000000,23.950000,24.620000
75%,0.000000,27.360000,29.320000
max,11.805900,33.410000,40.500000


 ### Usando "iloc" y el número de las columnas

In [19]:
datos.iloc[:,0:3].describe()

,precipIntensity,temperature,apparentTemperature
count,3361.000000,8640.000000,8640.000000
mean,0.027658,24.535807,25.664956
std,0.368538,3.388262,4.159469
min,0.000000,15.600000,15.640000
25%,0.000000,21.730000,22.420000
50%,0.000000,23.950000,24.620000
75%,0.000000,27.360000,29.320000
max,11.805900,33.410000,40.500000


 ## Selección de filas
 Veremos tres formas de hacerlo
 ### Usando iloc y el número de la fila

In [20]:
datos.iloc[0]

precipIntensity         0.00
temperature            22.44
apparentTemperature    22.73
dewPoint               18.10
humidity                0.77
pressure                 NaN
windSpeed               0.00
cloudCover              0.19
Name: 2018-01-01 06:00:00, dtype: float64

In [21]:
datos.iloc[-1]

precipIntensity          NaN
temperature            20.13
apparentTemperature    20.57
dewPoint               18.61
humidity                0.91
pressure                 NaN
windSpeed               3.60
cloudCover              0.00
Name: 2019-01-01 05:00:00, dtype: float64

In [22]:
type(datos.iloc[-1])

pandas.core.series.Series

 ### Usando el índice de la fila

In [23]:
datos['2018-01-01 06:00:00']

,precipIntensity,temperature,apparentTemperature,dewPoint,humidity,pressure,windSpeed,cloudCover
time,,,,,,,,
2018-01-01 06:00:00,0.0,22.44,22.73,18.1,0.77,NaN,0.0,0.19


In [24]:
type(datos['2018-01-01 06:00:00'])

pandas.core.frame.DataFrame

 ## Seleccionando múltiples filas

In [25]:
datos.iloc[0:3]

,precipIntensity,temperature,apparentTemperature,dewPoint,humidity,pressure,windSpeed,cloudCover
time,,,,,,,,
2018-01-01 06:00:00,0.0,22.44,22.73,18.10,0.77,NaN,0.0,0.19
2018-01-01 07:00:00,0.0,21.88,22.15,17.75,0.77,NaN,0.0,NaN
2018-01-01 08:00:00,0.0,21.33,21.63,17.94,0.81,NaN,0.0,NaN


 ## Seleccionando una porción del dataframe
 Veremos dos formas:
 ### Usando iloc

In [26]:
datos.iloc[[0,3,6,-1], [0,5,-3]] 

,precipIntensity,pressure,pressure
time,,,
2018-01-01 06:00:00,0.0,NaN,NaN
2018-01-01 09:00:00,0.0,NaN,NaN
2018-01-01 12:00:00,0.0,1014.9,1014.9
2019-01-01 05:00:00,NaN,NaN,NaN


 ### Usando corchetes e iloc

In [27]:
datos[['precipIntensity','pressure','pressure']].iloc[[0,3,6,-1]]

,precipIntensity,pressure,pressure
time,,,
2018-01-01 06:00:00,0.0,NaN,NaN
2018-01-01 09:00:00,0.0,NaN,NaN
2018-01-01 12:00:00,0.0,1014.9,1014.9
2019-01-01 05:00:00,NaN,NaN,NaN


 ## Cambiando el nombre de las columnas
 Los nombres actuales de las columnas son:

In [28]:
datos.columns

Index(['precipIntensity', 'temperature', 'apparentTemperature', 'dewPoint',
       'humidity', 'pressure', 'windSpeed', 'cloudCover'],
      dtype='object')

 ### Poniendo en minúsculas todos los nombres de columnas

In [29]:
datos = datos.rename(columns=str.lower) 
datos.columns

# datos.rename(columns=str.lower, inplace = True)

Index(['precipintensity', 'temperature', 'apparenttemperature', 'dewpoint',
       'humidity', 'pressure', 'windspeed', 'cloudcover'],
      dtype='object')

In [30]:
datos = datos.rename(
    columns = {
        'precipintensity': 'precip_intensity',
        'apparenttemperature': 'apparent_temperature',
        'dewpoint': 'dew_point',
        'windspeed': 'wind_speed',
        'cloudcover': 'cloud_cover'
    }
)
datos.columns

Index(['precip_intensity', 'temperature', 'apparent_temperature', 'dew_point',
       'humidity', 'pressure', 'wind_speed', 'cloud_cover'],
      dtype='object')

 ## Escritura de archivos excel
 ### Sin formato

In [31]:
reporte = dir_inter / 'clima2018.xlsx'
writer = pd.ExcelWriter(reporte, engine='xlsxwriter')
datos.to_excel(writer,'Datos', index=True)
writer.save()

 ### Con formato

In [32]:
reporte = dir_inter / 'clima2018_format.xlsx'
writer = pd.ExcelWriter(reporte, engine='xlsxwriter')
workbook  = writer.book
datos.to_excel(writer,'Datos', index=True)
worksheet = writer.sheets['Datos']
#Creamos los formatos condicionales
format1 = workbook.add_format({'bg_color':   '#FFC7CE',
                            'font_color': '#9C0006'})
format2 = workbook.add_format({'bg_color':   '#C6EFCE',
                                       'font_color': '#006100'})
#Aplicamos los formatos
rango1 = 'B1:I' + str(len(datos) + 1)
worksheet.conditional_format(rango1, {'type': 'blanks',
                                        'format':   format1})   
rango2 = 'B1:I' + str(len(datos) + 1)   
worksheet.conditional_format(rango2, {'type':     'cell',
                                       'criteria': 'between',
                                       'minimum':  25,
                                       'maximum':  28,
                                       'format':   format2}) 
writer.save()
#Autoajustar el ancho de la columna
#Librería xlwings
reporte2 = str(reporte)
wb = xw.Book(reporte2)
sht = wb.sheets['Datos']
sht.autofit('c')
wb.save(reporte2)

 #SPAM
 Muchas gracias, no te olvides de darle like, suscribirte y activar las notificaciones